In [1]:
import re
from IPython.display import display

import joblib
import numpy as np
import torch
import torch.utils.data as data_utils

from lolip.models.torch_utils.archs import *
from lolip.variables import auto_var
from lolip.utils import estimate_local_lip, estimate_local_lip_v2
from params import mnistLip, cifarLip
from utils import params_to_dataframe

In [3]:
_, _, grid_params, _ = mnistLip()()
df = params_to_dataframe(grid_params, columns=['trn_acc', 'tst_acc', 'adv_trn_acc', 'adv_tst_acc', 'avg_trn_lip', 'avg_tst_lip', 'history'], result_file_dir='./results/experiment01/')
for name, d in df.groupby("dataset"):
    display(d)

I1228 23:14:56.762578 140590221182784 auto_var.py:261] Running time: 0.006830
I1228 23:14:56.835733 140590221182784 auto_var.py:261] Running time: 0.002434
I1228 23:14:56.902734 140590221182784 auto_var.py:261] Running time: 0.002371
I1228 23:14:56.967411 140590221182784 auto_var.py:261] Running time: 0.002389
I1228 23:14:57.032747 140590221182784 auto_var.py:261] Running time: 0.002408
E1228 23:14:57.155629 140590221182784 auto_var.py:341] Error with {'attack': 'pgd', 'dataset': 'mnist', 'eps': 0.1, 'model': 'curece-tor-CNN001', 'norm': 'inf', 'random_seed': 0}
I1228 23:14:57.159345 140590221182784 auto_var.py:261] Running time: 0.002357
I1228 23:14:57.226985 140590221182784 auto_var.py:261] Running time: 0.002395
I1228 23:14:57.291795 140590221182784 auto_var.py:261] Running time: 0.002422
I1228 23:14:57.358804 140590221182784 auto_var.py:261] Running time: 0.002388


problem with ./results/experiment01/pgd-mnist-0.1-curece-tor-CNN001-inf-0.pkl


I1228 23:14:57.424295 140590221182784 auto_var.py:261] Running time: 0.002435
I1228 23:14:57.488916 140590221182784 auto_var.py:261] Running time: 0.002347
I1228 23:14:57.553661 140590221182784 auto_var.py:261] Running time: 0.002398
I1228 23:14:57.618907 140590221182784 auto_var.py:261] Running time: 0.002380
I1228 23:14:57.685503 140590221182784 auto_var.py:261] Running time: 0.002392
I1228 23:14:57.750255 140590221182784 auto_var.py:261] Running time: 0.002412
I1228 23:14:57.816773 140590221182784 auto_var.py:261] Running time: 0.002397
I1228 23:14:57.881052 140590221182784 auto_var.py:261] Running time: 0.002403
I1228 23:14:57.947025 140590221182784 auto_var.py:261] Running time: 0.002379
I1228 23:14:58.011559 140590221182784 auto_var.py:261] Running time: 0.002399
I1228 23:14:58.076672 140590221182784 auto_var.py:261] Running time: 0.002396
I1228 23:14:58.142613 140590221182784 auto_var.py:261] Running time: 0.002371
E1228 23:14:58.268203 140590221182784 auto_var.py:341] Error wit

problem with ./results/experiment01/pgd-fashion-0.1-curece-tor-ResNet50-inf-0.pkl
problem with ./results/experiment01/pgd-fashion-0.1-advce-tor-ResNet50-inf-0.pkl
problem with ./results/experiment01/pgd-fashion-0.1-llr65ce-tor-ResNet50-inf-0.pkl


,attack,dataset,eps,model,norm,random_seed,trn_acc,tst_acc,adv_trn_acc,adv_tst_acc,avg_trn_lip,avg_tst_lip,history
9,pgd,fashion,0.1,ce-tor-CNN001,inf,0,0.999533,0.9263,0.008750,0.0558,76.034775,76.180283,"[{'epoch': 1, 'trn_loss': 0.6656579953034719, ..."
10,pgd,fashion,0.1,tradesce-tor-CNN001,inf,0,0.996783,0.9308,0.246233,0.2813,22.103630,21.940477,"[{'epoch': 1, 'trn_loss': 0.703578668832779, '..."
11,pgd,fashion,0.1,trades6ce-tor-CNN001,inf,0,0.976017,0.9194,0.503983,0.5260,8.076009,8.072925,"[{'epoch': 1, 'trn_loss': 0.7347536305745442, ..."
12,pgd,fashion,0.1,trades6K20ce-tor-CNN001,inf,0,0.975017,0.9216,0.498300,0.5156,7.568070,7.581863,"[{'epoch': 1, 'trn_loss': 0.7420752228101094, ..."
13,pgd,fashion,0.1,trades10ce-tor-CNN001,inf,0,0.963883,0.9130,0.571000,0.5846,5.593621,5.636191,"[{'epoch': 1, 'trn_loss': 0.7621811085700989, ..."
14,pgd,fashion,0.1,curece-tor-CNN001,inf,0,1.000000,0.9214,0.076683,0.1263,28.173588,27.971794,"[{'epoch': 1, 'trn_loss': 0.6511157069206238, ..."
15,pgd,fashion,0.1,advce-tor-CNN001,inf,0,0.888167,0.8686,0.829900,0.8119,1.136331,1.133796,"[{'epoch': 1, 'trn_loss': 1.1296655715942383, ..."
16,pgd,fashion,0.1,llr65ce-tor-CNN001,inf,0,0.955950,0.9109,0.435117,0.4444,12.036719,12.132747,"[{'epoch': 1, 'trn_loss': 5.315398379262288, '..."
17,pgd,fashion,0.1,llrce-tor-CNN001,inf,0,0.975550,0.9157,0.393817,0.4215,9.833105,9.933347,"[{'epoch': 1, 'trn_loss': 3.6845599964141846, ..."
18,pgd,fashion,0.1,gr4ce-tor-CNN001,inf,0,0.999633,0.9288,0.012767,0.0592,83.943481,83.509087,"[{'epoch': 1, 'trn_loss': 0.6925393321673076, ..."


,attack,dataset,eps,model,norm,random_seed,trn_acc,tst_acc,adv_trn_acc,adv_tst_acc,avg_trn_lip,avg_tst_lip,history
0,pgd,mnist,0.1,ce-tor-CNN001,inf,0,1.000000,0.9920,0.408133,0.4183,24.914808,25.083706,"[{'epoch': 1, 'trn_loss': 0.4666699381828308, ..."
1,pgd,mnist,0.1,tradesce-tor-CNN001,inf,0,1.000000,0.9922,0.806650,0.8088,6.701021,6.729030,"[{'epoch': 1, 'trn_loss': 0.46601010074218113,..."
2,pgd,mnist,0.1,trades6ce-tor-CNN001,inf,0,0.999967,0.9924,0.919833,0.9062,5.737867,5.748982,"[{'epoch': 1, 'trn_loss': 0.4758203192869822, ..."
3,pgd,mnist,0.1,trades6K20ce-tor-CNN001,inf,0,0.999983,0.9926,0.902850,0.8871,5.639750,5.641222,"[{'epoch': 1, 'trn_loss': 0.480186371076107, '..."
4,pgd,mnist,0.1,trades10ce-tor-CNN001,inf,0,0.999967,0.9931,0.921550,0.9043,5.052001,5.040387,"[{'epoch': 1, 'trn_loss': 0.4963904350121816, ..."
5,pgd,mnist,0.1,advce-tor-CNN001,inf,0,0.999817,0.9931,0.996400,0.9670,2.771575,2.795055,"[{'epoch': 1, 'trn_loss': 0.7555228825410207, ..."
6,pgd,mnist,0.1,llr65ce-tor-CNN001,inf,0,0.999400,0.9916,0.839967,0.8302,13.091495,13.002464,"[{'epoch': 1, 'trn_loss': 2.5009387490272523, ..."
7,pgd,mnist,0.1,llrce-tor-CNN001,inf,0,0.999983,0.9934,0.860450,0.8399,12.434842,12.424883,"[{'epoch': 1, 'trn_loss': 1.71617512169679, 't..."
8,pgd,mnist,0.1,gr4ce-tor-CNN001,inf,0,0.999983,0.9913,0.416417,0.4312,19.298149,19.351391,"[{'epoch': 1, 'trn_loss': 0.4542612582564354, ..."


In [5]:
_, _, grid_params, _ = cifarLip()()
print(grid_params)
params_to_dataframe(grid_params, columns=['trn_acc', 'history'], result_file_dir='./results/experiment01/')

I1228 12:24:11.952974 139870957287232 auto_var.py:261] Running time: 0.000193
I1228 12:24:12.026747 139870957287232 auto_var.py:261] Running time: 0.000348
I1228 12:24:12.094060 139870957287232 auto_var.py:261] Running time: 0.000043


[{'dataset': ['cifar10'], 'model': ['ce-tor-WRN_40_10', 'trades6ce-tor-WRN_40_10', 'curece-tor-WRN_40_10', 'advce-tor-WRN_40_10', 'llrce-tor-WRN_40_10'], 'eps': [0.031], 'norm': ['inf'], 'attack': ['pgd'], 'random_seed': [0]}]


I1228 12:24:12.161438 139870957287232 auto_var.py:261] Running time: 0.000042
I1228 12:24:12.229044 139870957287232 auto_var.py:261] Running time: 0.000040


ValueError: not enough values to unpack (expected 2, got 0)

In [9]:
def get_arch(path):
    return path.split("-")[path.split("-").index('tor')+1]

def get_ds(path):
    return path.split("-")[1]

def predict_real(model, X):
    X = X.transpose(0, 3, 1, 2)
    dataset = data_utils.TensorDataset(torch.from_numpy(X).float())
    loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=False, num_workers=2)
    ret = []
    for [x] in loader:
        ret.append(model(x.to('cuda')).detach().cpu().numpy())
    return np.concatenate(ret, axis=0)

def calc_lip(model, X, Xp, top_norm, btm_norm):
    top = np.linalg.norm(predict_real(model, X)-predict_real(model, Xp), ord=top_norm, axis=1)
    down = np.linalg.norm(X.reshape(len(Xp), -1)-Xp.reshape(len(Xp), -1), ord=btm_norm, axis=1)
    return top / (down+1e-6)

def get_lip(model, X, top_norm=2, btm_norm=2, epsilon=1.0):
    lip = estimate_local_lip_v2(model, X, top_norm=top_norm, btm_norm=btm_norm, epsilon=epsilon)
    lip_ = calc_lip(model, X, lip, top_norm=top_norm, btm_norm=btm_norm)
    return lip_

class ModelWrapper():
    def __init__(self, model):
        self.model = model
    def predict_real(self, X):
        return predict_real(self.model, X)
    def predict(self, X):
        return predict_real(self.model, X).argmax(axis=1)

In [10]:
model_path = "./models/pgd-mnist-0.031-ce-tor-CNN001-inf-0.pt"
model = globals()[get_arch(model_path)](n_classes=10)
model.load_state_dict(torch.load(model_path))
model.eval()
model.cuda()

trnX, trny, tstX, tsty = auto_var.get_var_with_argument("dataset", get_ds(model_path))
lip_ = get_lip(model, trnX, btm_norm=np.inf, epsilon=0.03)
print(lip_.mean())

30.94554163004557
30.943932


In [11]:
trny

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [32]:
lip_ = get_lip(model, trnX)

0.0775341446240743


In [4]:
np.attacks.torch.projected_gradient_descent import ProjectedGradientDescent
nb_iter=10
attack_model = ProjectedGradientDescent(
    model_fn=model,
    norm=np.inf,
    eps=0.1,
    eps_iter=0.1*2/nb_iter,
    nb_iter=nb_iter,
)

adv_trnX = attack_model.perturb(trnX)
adv_tstX = attack_model.perturb(tstX)
print((ModelWrapper(model).predict(trnX) == trny).mean())
print((ModelWrapper(model).predict(adv_trnX) == trny).mean())

SyntaxError: invalid syntax (<ipython-input-4-319c90070ab9>, line 1)

In [17]:
ModelWrapper(model).predict(trnX)

array([5, 0, 4, ..., 5, 6, 8])

In [3]:
model(torch.Tensor(trnX[0:10].transpose(0, 3, 1, 2)))

NameError: name 'model' is not defined